In [26]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
import random
from icecream import ic
from tqdm.auto import tqdm
from matplotlib import pyplot as plt
from itertools import accumulate

In [27]:
logging.basicConfig(level=logging.DEBUG)

## Nearest Neighbor (Greedy)

In [28]:
def nearest_neighbor_tsp(start_city_index=0):
    """
    Risolve il TSP utilizzando l'algoritmo Greedy di Nearest Neighbor.
    :param start_city_index: Indice della città di partenza
    :return: Lista dell'ordine delle città visitate e lunghezza totale del percorso
    """
    dist_matrix=DIST_MATRIX.copy()
    visited = np.full(len(CITIES), False)  # Array per tenere traccia delle città visitate
    city = start_city_index  
    visited[city] = True

    tsp = [city]  # Lista per mantenere l'ordine delle città visitate

    # Loop fino a quando tutte le città non sono state visitate
    while not np.all(visited):
        # Imposta le distanze delle città già visitate a infinito per escluderle dalla ricerca del più vicino
        dist_matrix[:, city] = np.inf
        # Trova la città più vicina che non è ancora stata visitata
        closest_city = np.argmin(dist_matrix[city])
        # Marca la città trovata come visitata
        visited[closest_city] = True
        # Aggiorna l'ordine delle città visitate
        tsp.append(int(closest_city))
        # Aggiorna la città corrente
        city = closest_city

    # Ritorna alla città di partenza
    tsp.append(start_city_index)
    
    total_distance = tsp_cost(tsp,DIST_MATRIX)
    
    return tsp, total_distance

In [29]:
def valid(tsp):
    """
    Verifica la validità di un percorso TSP.
    """
    tsp = np.array(tsp)
    
    if tsp[0] != tsp[-1]:
        return False
    
    # Verifica che tutte le città siano visitate esattamente una volta
    cities_visited = tsp[:-1]
    expected_cities = np.arange(len(CITIES))
    
    return (len(np.unique(cities_visited)) == len(CITIES) and 
            np.all(np.isin(expected_cities, cities_visited)))

def tsp_cost(route, dist_matrix):
    """Calcola il costo del percorso e la sua validità."""
    route = np.array(route)
    # Calcola le distanze tra città consecutive
    total_distance = np.sum([dist_matrix[route[i], route[i+1]] 
                           for i in range(len(route)-1)])
    return (total_distance, valid(route))

## Inversion mutation

In [30]:
def tweak(tsp):
    """Esegue un'inversion mutation su una soluzione TSP"""
    tsp = np.array(tsp)
    # Seleziona due punti casuali nel percorso per eseguire l'inversione
    a, b = sorted(np.random.choice(range(1, len(tsp) - 1), 2, replace=False))
     # Inverte il sotto-percorso tra i due punti selezionati
    tsp[a:b+1] = np.flip(tsp[a:b+1])
    return tsp.tolist()


##  Simulated Annealing (SA) by Initial Population.

In [31]:
def simulated_annealing_population(dist_matrix, initial_temp, cooling_rate, stop_temp, population_size=100):
    """Algoritmo di Simulated Annealing per generare una popolazione iniziale"""
    dist_matrix = np.array(dist_matrix)
    # Trova una soluzione iniziale con il metodo Greedy
    current_solution, current_cost = nearest_neighbor_tsp(start_city_index=0)
    best_solution = current_solution[:]
    best_cost = current_cost

    population = [(best_solution, best_cost)]
    temperature = initial_temp

    while temperature > stop_temp:
        new_solution = tweak(current_solution[:])
        new_cost = tsp_cost(new_solution, dist_matrix)

        if (new_cost[0] < current_cost[0] or 
            np.random.random() < np.exp((current_cost[0] - new_cost[0]) / temperature)):
            current_solution = new_solution
            current_cost = new_cost

            if new_cost[0] < best_cost[0]:
                best_solution = new_solution
                best_cost = new_cost
    
            population.append((current_solution, current_cost))
            
        temperature *= cooling_rate
    # Ordina la popolazione per costo e restituisci i migliori 'population_size' individui    
    population = np.array(population, dtype=object)
    sorted_indices = np.argsort([cost[0] for _, cost in population])
    population = population[sorted_indices]
    
    return population[:population_size].tolist()

In [32]:
def initial_population(pop_size):
    """Inizializza la popolazione usando parametri predefiniti."""
    initial_temp = 1000
    cooling_rate = 0.992
    stop_temp = 1e-200
    # Genera la popolazione iniziale con Simulated Annealing
    return simulated_annealing_population(DIST_MATRIX, initial_temp,
                                        cooling_rate, stop_temp, pop_size)


##  Crossover Inver-Over

In [33]:
def inver_over_crossover(parent1, parent2, num_iterations=1):
    """Applica il crossover inver-over """
    parent1, parent2 = np.array(parent1), np.array(parent2)
    child = parent1.copy()
 # Esegui una serie di iterazioni di crossover per creare il figlio
    for _ in range(num_iterations):
        start_index = np.random.randint(1, len(child) - 2)
        start_city = child[start_index]
         # Seleziona casualmente una città di destinazione da uno dei genitori
        if np.random.random() < 0.5:
            valid_cities = child[1:-1][child[1:-1] != start_city]
            end_city = np.random.choice(valid_cities)
        else:
            valid_cities = parent2[1:-1][parent2[1:-1] != start_city]
            end_city = np.random.choice(valid_cities)
        # Ottieni le posizioni della città di partenza e di destinazione nel figlio
        start_pos = np.where(child == start_city)[0][0]
        end_pos = np.where(child == end_city)[0][0]
        
        if start_pos > end_pos:
            start_pos, end_pos = end_pos, start_pos
          # Inverti il sotto-percorso tra le due posizioni
        child[start_pos:end_pos + 1] = np.flip(child[start_pos:end_pos + 1])
    # Assicurati che il figlio termini nella stessa città di partenza
    if child[0] != child[-1]:
        child = np.append(child, child[0])
    
    return child.tolist()

In [34]:
def tournament_selection(population, k=5):
    """Selezione torneo usando NumPy."""
     # Seleziona casualmente 'k' individui dalla popolazione
    selected_indices = np.random.choice(len(population), k, replace=False)
    selected = np.array(population, dtype=object)[selected_indices]
     # Ritorna l'individuo con il costo minore tra quelli selezionati
    return min(selected, key=lambda x: x[1][0])  # x[1][0] accede al costo


## Inversion mutation

In [35]:
def inversion_mutation(tsp):
    """Esegue un'inversion mutation """
    tsp = np.array(tsp)
    a, b = sorted(np.random.choice(range(1, len(tsp) - 1), 2, replace=False))
    tsp[a:b+1] = np.flip(tsp[a:b+1])
    return tsp.tolist()


## EA with adaptive mutation and multiple restart with simulated anneling.

In [36]:


def genetic_algorithm_tsp_adaptive_mutation(
    
    dist_matrix,
    population_size=100, 
    tournament_size=10, 
    num_generations=10000, 
    initial_mutation_rate=0.1, 
    crossover_iterations=1,
    max_generations_without_improvement=500,
    mutation_increase_factor=1.5,
    mutation_decrease_factor=0.7,
    mutation_threshold=1.3,  # Soglia per il riavvio
    initial_temp=1000,      
    cooling_rate=0.992,
    stop_temp=1e-200,

):
    """Risolve il TSP utilizzando un Algoritmo Genetico con mutazione adattiva."""
    dist_matrix = np.array(dist_matrix)
    population = initial_population(population_size)
    current_mutation_rate = initial_mutation_rate

    best_solution, best_cost = min(population, key=lambda x: x[1][0])
    all_time_best_solution = best_solution
    all_time_best_cost = best_cost
    generations_without_improvement = 0
    

    for generation in tqdm(range(num_generations)):
        # Controlla se è necessario riavviare con SA
        if current_mutation_rate > mutation_threshold:
            print(f"\nMutation rate ({current_mutation_rate:.3f}) exceeded threshold ({mutation_threshold})")
            print("Restarting with Simulated Annealing...")
            
            # Esegui SA e ottieni nuova popolazione
            new_population = simulated_annealing_population(
                dist_matrix, 
                initial_temp, 
                cooling_rate, 
                stop_temp, 
                population_size
            )
            population = sorted(new_population, key=lambda x: x[1][0])[:population_size]
            # Reset del tasso di mutazione
            current_mutation_rate = initial_mutation_rate
            print("Population refreshed and mutation rate reset")

        new_population = []

        while len(new_population) < population_size:
            parent1 = tournament_selection(population, tournament_size)
            parent2 = tournament_selection(population, tournament_size)
            
            child = inver_over_crossover(parent1[0], parent2[0], crossover_iterations)
            
            if np.random.random() < current_mutation_rate:
                child = inversion_mutation(child)

            child_cost = tsp_cost(child, dist_matrix)
            new_population.append((child, child_cost))

        combined_population = np.array(new_population + population, dtype=object)
        sorted_indices = np.argsort([cost[0] for _, cost in combined_population])
        population = combined_population[sorted_indices][:population_size].tolist()

        new_best_solution, new_best_cost = min(population, key=lambda x: x[1][0])

        # Aggiorna il miglior risultato di sempre se necessario
        if new_best_cost[0] < all_time_best_cost[0]:
            all_time_best_solution = new_best_solution
            all_time_best_cost = new_best_cost

        if new_best_cost[0] < best_cost[0]:
            best_solution, best_cost = new_best_solution, new_best_cost
            generations_without_improvement = 0
            current_mutation_rate = max(initial_mutation_rate, 
                                     current_mutation_rate * mutation_decrease_factor)
        else:
            generations_without_improvement += 1

        if generations_without_improvement > max_generations_without_improvement:
            current_mutation_rate *= mutation_increase_factor
            generations_without_improvement = 0

        if generation % 1000 == 0:
            print(f"Generazione {generation}: "
                  f"Costo migliore corrente = {best_cost[0]:.2f} | "
                  f"Costo migliore assoluto = {all_time_best_cost[0]:.2f} | "
                  f"Tasso di mutazione = {current_mutation_rate:.3f} | "
                  f"Valido = {best_cost[1]}")
    
    return all_time_best_solution, all_time_best_cost


## Vanuatu

In [37]:

CITIES = pd.read_csv('./cities/vanuatu.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
genetic_algorithm_tsp_adaptive_mutation(
    DIST_MATRIX,
    num_generations=0, 
)



0it [00:00, ?it/s]


([0, 7, 1, 4, 3, 5, 6, 2, 0], (np.float64(1345.5449564733112), np.True_))

For vanuatu just applying simulated anneling to get the best solution is enough

## Italy

In [42]:

CITIES = pd.read_csv('./cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
genetic_algorithm_tsp_adaptive_mutation(
    DIST_MATRIX,
    mutation_increase_factor=1.3,
    mutation_decrease_factor=0.7
)

  0%|          | 16/10000 [00:00<02:08, 77.69it/s]

Generazione 0: Costo migliore corrente = 4414.43 | Costo migliore assoluto = 4414.43 | Tasso di mutazione = 0.100 | Valido = True


 10%|█         | 1016/10000 [00:14<01:55, 77.73it/s]

Generazione 1000: Costo migliore corrente = 4181.62 | Costo migliore assoluto = 4181.62 | Tasso di mutazione = 0.130 | Valido = True


 20%|██        | 2009/10000 [00:28<01:51, 71.39it/s]

...

Generazione 8000: Costo migliore corrente = 4172.76 | Costo migliore assoluto = 4172.76 | Tasso di mutazione = 0.100 | Valido = True


 90%|█████████ | 9013/10000 [02:10<00:13, 74.93it/s]

Generazione 9000: Costo migliore corrente = 4172.76 | Costo migliore assoluto = 4172.76 | Tasso di mutazione = 0.169 | Valido = True


100%|██████████| 10000/10000 [02:24<00:00, 69.33it/s]


([0,
  27,
  26,
  39,
  34,
  15,
  14,
  21,
  35,
  11,
  1,
  2,
  38,
  17,
  31,
  37,
  8,
  24,
  7,
  36,
  16,
  10,
  29,
  4,
  19,
  32,
  25,
  28,
  13,
  42,
  22,
  18,
  20,
  3,
  6,
  44,
  45,
  40,
  5,
  41,
  43,
  23,
  9,
  30,
  12,
  33,
  0],
 (np.float64(4172.7626139164095), np.True_))

## Russia

In [39]:

CITIES = pd.read_csv('./cities/russia.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
genetic_algorithm_tsp_adaptive_mutation(DIST_MATRIX)

  0%|          | 4/10000 [00:00<04:19, 38.57it/s]

Generazione 0: Costo migliore corrente = 35895.96 | Costo migliore assoluto = 35895.96 | Tasso di mutazione = 0.100 | Valido = True


 10%|█         | 1005/10000 [00:22<03:12, 46.82it/s]

Generazione 1000: Costo migliore corrente = 35659.46 | Costo migliore assoluto = 35659.46 | Tasso di mutazione = 0.100 | Valido = True


 20%|██        | 2006/10000 [00:44<02:46, 48.15it/s]

...

Generazione 8000: Costo migliore corrente = 35216.50 | Costo migliore assoluto = 35216.50 | Tasso di mutazione = 0.100 | Valido = True


 90%|█████████ | 9007/10000 [03:19<00:17, 56.12it/s]

Generazione 9000: Costo migliore corrente = 35194.62 | Costo migliore assoluto = 35194.62 | Tasso di mutazione = 0.100 | Valido = True


100%|██████████| 10000/10000 [03:37<00:00, 45.89it/s]


([0,
  54,
  1,
  40,
  139,
  124,
  90,
  58,
  109,
  86,
  15,
  11,
  113,
  97,
  137,
  142,
  55,
  36,
  159,
  105,
  49,
  20,
  65,
  166,
  143,
  131,
  117,
  62,
  101,
  91,
  100,
  99,
  96,
  118,
  85,
  134,
  138,
  25,
  145,
  119,
  136,
  103,
  9,
  120,
  29,
  37,
  155,
  153,
  28,
  8,
  24,
  38,
  63,
  42,
  30,
  73,
  149,
  72,
  111,
  45,
  22,
  76,
  130,
  5,
  64,
  128,
  88,
  53,
  135,
  12,
  112,
  84,
  152,
  125,
  89,
  13,
  56,
  129,
  156,
  59,
  160,
  102,
  18,
  127,
  147,
  34,
  110,
  148,
  47,
  116,
  107,
  67,
  123,
  4,
  133,
  44,
  51,
  158,
  154,
  21,
  115,
  141,
  121,
  69,
  50,
  126,
  10,
  164,
  60,
  66,
  43,
  163,
  95,
  108,
  122,
  94,
  35,
  140,
  87,
  114,
  46,
  165,
  27,
  81,
  98,
  150,
  32,
  52,
  68,
  7,
  26,
  80,
  19,
  83,
  161,
  39,
  2,
  77,
  70,
  74,
  33,
  104,
  14,
  79,
  75,
  132,
  78,
  93,
  92,
  82,
  17,
  157,
  61,
  106,
  162,
  48,
  41,
  

## Us

In [40]:

CITIES = pd.read_csv('./cities/us.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
genetic_algorithm_tsp_adaptive_mutation(DIST_MATRIX)

  0%|          | 3/10000 [00:00<06:23, 26.04it/s]

Generazione 0: Costo migliore corrente = 43138.94 | Costo migliore assoluto = 43138.94 | Tasso di mutazione = 0.100 | Valido = True


 10%|█         | 1008/10000 [00:27<03:50, 39.01it/s]

Generazione 1000: Costo migliore corrente = 42026.63 | Costo migliore assoluto = 42026.63 | Tasso di mutazione = 0.100 | Valido = True


 20%|██        | 2008/10000 [00:53<03:32, 37.62it/s]

...

Generazione 8000: Costo migliore corrente = 40505.92 | Costo migliore assoluto = 40505.92 | Tasso di mutazione = 0.100 | Valido = True


 90%|█████████ | 9005/10000 [04:37<00:25, 38.61it/s]

Generazione 9000: Costo migliore corrente = 40490.06 | Costo migliore assoluto = 40490.06 | Tasso di mutazione = 0.100 | Valido = True


100%|██████████| 10000/10000 [05:09<00:00, 32.36it/s]


([0,
  162,
  6,
  205,
  197,
  33,
  303,
  320,
  300,
  206,
  211,
  152,
  129,
  139,
  156,
  112,
  97,
  29,
  297,
  317,
  13,
  145,
  73,
  43,
  121,
  58,
  232,
  242,
  2,
  177,
  204,
  86,
  149,
  302,
  108,
  172,
  44,
  295,
  271,
  223,
  289,
  119,
  282,
  150,
  200,
  288,
  216,
  87,
  308,
  258,
  239,
  96,
  236,
  226,
  208,
  138,
  243,
  184,
  170,
  186,
  294,
  88,
  81,
  51,
  260,
  39,
  203,
  63,
  132,
  265,
  65,
  127,
  106,
  209,
  8,
  104,
  315,
  85,
  78,
  76,
  201,
  158,
  301,
  131,
  159,
  36,
  109,
  217,
  215,
  146,
  267,
  274,
  298,
  212,
  259,
  268,
  19,
  310,
  56,
  102,
  255,
  263,
  266,
  287,
  101,
  118,
  264,
  202,
  67,
  262,
  24,
  241,
  306,
  269,
  92,
  305,
  11,
  182,
  285,
  84,
  250,
  248,
  237,
  277,
  50,
  89,
  254,
  123,
  228,
  115,
  307,
  291,
  95,
  140,
  238,
  23,
  91,
  272,
  125,
  9,
  278,
  279,
  187,
  171,
  27,
  256,
  319,
  316,
  231,
 

## China

In [41]:

CITIES = pd.read_csv('./cities/china.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
genetic_algorithm_tsp_adaptive_mutation(
    DIST_MATRIX,
    num_generations=20000, 
)

  0%|          | 3/10000 [00:00<08:05, 20.57it/s]

Generazione 0: Costo migliore corrente = 63962.92 | Costo migliore assoluto = 63962.92 | Tasso di mutazione = 0.100 | Valido = True


 10%|█         | 1004/10000 [00:58<07:49, 19.16it/s]

Generazione 1000: Costo migliore corrente = 61298.43 | Costo migliore assoluto = 61298.43 | Tasso di mutazione = 0.100 | Valido = True


 20%|██        | 2003/10000 [01:50<06:03, 21.99it/s]

...

Generazione 8000: Costo migliore corrente = 56324.02 | Costo migliore assoluto = 56324.02 | Tasso di mutazione = 0.100 | Valido = True


 90%|█████████ | 9004/10000 [08:15<00:56, 17.77it/s]

Generazione 9000: Costo migliore corrente = 56113.07 | Costo migliore assoluto = 56113.07 | Tasso di mutazione = 0.100 | Valido = True


100%|██████████| 10000/10000 [09:02<00:00, 18.43it/s]


([0,
  186,
  505,
  570,
  481,
  414,
  173,
  396,
  397,
  198,
  290,
  101,
  451,
  395,
  233,
  23,
  507,
  240,
  191,
  133,
  544,
  644,
  399,
  539,
  516,
  174,
  28,
  573,
  192,
  239,
  154,
  382,
  685,
  235,
  172,
  629,
  681,
  358,
  450,
  232,
  412,
  207,
  696,
  5,
  457,
  469,
  356,
  554,
  16,
  532,
  434,
  435,
  514,
  699,
  506,
  545,
  33,
  139,
  34,
  59,
  340,
  66,
  61,
  724,
  346,
  531,
  183,
  530,
  143,
  535,
  309,
  218,
  716,
  123,
  388,
  52,
  509,
  470,
  637,
  549,
  31,
  688,
  603,
  553,
  277,
  206,
  24,
  128,
  88,
  510,
  666,
  623,
  634,
  668,
  282,
  528,
  168,
  262,
  237,
  370,
  129,
  335,
  211,
  180,
  542,
  22,
  185,
  560,
  92,
  90,
  26,
  310,
  161,
  361,
  537,
  169,
  426,
  630,
  672,
  120,
  342,
  576,
  572,
  176,
  503,
  89,
  647,
  339,
  585,
  439,
  25,
  705,
  19,
  102,
  312,
  347,
  612,
  580,
  267,
  692,
  288,
  258,
  673,
  115,
  179,
  3,
  5